In [2]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [37]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rathore/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [38]:
api_key = "pincone_api"

In [39]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [40]:
import os
from pinecone import Pinecone, ServerlessSpec

index_name = "hybrid-search-langchain-pinecone"

# initialize pinecone client
pc = Pinecone(api_key=api_key)

# create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, #dimension of dense vector
        metric= "dotproduct", #sparse value supported only for dot product
        spec= ServerlessSpec(cloud= "aws", region="us-east-1"),
    )

In [41]:
index = pc.Index(index_name)
index

In [44]:
## vector embedding

import os 
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [45]:
from pinecone_text.sparse import BM25Encoder # for sparse matrix 

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [46]:
sentences= [
    "In 2023, I visited Paris",
        "In 2022, I visited New york",
        "In 2021, I visted Egypt",
]

## Applysing tf-idf values of the sentences
bm25_encoder.fit(sentences)

## store the value to JSON file

bm25_encoder.dump("bm25_values.json")

## load to BM25 enocider object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 101.99it/s]


In [27]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [28]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x3195cb010>, index=<pinecone.data.index.Index object at 0x10a2dbd00>)

In [47]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
        "In 2022, I visited New york",
        "In 2021, I visted Egypt",
]
)

100%|██████████| 1/1 [00:03<00:00,  3.46s/it]


In [49]:
retriever.invoke("What city did I visit last?")

[Document(metadata={'score': 0.382815}, page_content='In 2022, I visited New york'),
 Document(metadata={'score': 0.360170245}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.150362492}, page_content='In 2021, I visted Egypt')]